In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

from source_code.SqlCo import Sqldd

Autosave disabled
Local Sec Env loaded


In [2]:
## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
230,ABW,North America,Aruba,2020-11-03,4524.0,5.0,14.857,38.0,0.0,0.286,...,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
539,AFG,Asia,Afghanistan,2020-11-03,41728.0,95.0,99.429,1544.0,3.0,3.143,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
766,AGO,Africa,Angola,2020-11-03,11035.0,230.0,236.286,286.0,2.0,2.571,...,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN,61.15,0.581


In [3]:
## dump the file for using it in other nb
# c.to_csv(f"data\\Base_Files\\full_data{date.today()}.csv", index=False)

In [4]:
## Load data from days -1 to evaluate the performances of model
yesterday_date = date.today() - timedelta(days=1)
yesterday_date = str(yesterday_date)

tip = Sqldd()
cnx, cursor = tip.get_bdd_co()
data_load = pd.read_sql(f"SELECT * FROM cov_aipred WHERE date='{yesterday_date}';", con=cnx) 

In [5]:
def Eval(c,data_load,country):
    """ function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    res_tempo= res_tempo.reset_index()
    rez1 = res_tempo.at[0,"total_cases_predict"]
    rez2 = res_tempo.at[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.at[0,"date"] = v0
    follow_df.at[0,"country"] = country
    follow_df.at[0,"total_cases_predict"] = rez1
    follow_df.at[0,"total_cases_real"] = v1
    follow_df.at[0,"total_deaths_predict"] = rez2
    follow_df.at[0,"total_deaths_real"] = v2
    follow_df.at[0,"error_abs_cases"] = errorCase1
    follow_df.at[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [6]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Belgium")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [7]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-11-03,France,1451560.0,1466433.0,37197.0,37435.0,-14873.0,-238.0
0,2020-11-03,China,91366.0,91396.0,4743.0,4739.0,-30.0,4.0
0,2020-11-03,Italy,743573.0,731588.0,39069.0,39059.0,11985.0,10.0
0,2020-11-02,Spain,822383.0,1240697.0,32245.0,36257.0,-418314.0,-4012.0
0,2020-11-03,United States,9269360.0,9291245.0,232256.0,231551.0,-21885.0,705.0
0,2020-11-03,World,47083300.0,47093222.0,1210360.0,1207290.0,-9922.0,3070.0
0,2020-11-03,United Kingdom,1056900.0,1053864.0,46985.0,46853.0,3036.0,132.0
0,2020-11-03,Germany,558537.0,560379.0,10556.0,10661.0,-1842.0,-105.0
0,2020-11-03,Iran,622177.0,628780.0,35448.0,35738.0,-6603.0,-290.0
0,2020-11-03,Turkey,378831.0,379775.0,10388.0,10402.0,-944.0,-14.0


In [8]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
# rapport.to_csv(f"data\\Rapport\\rap{date.today()}.csv")

In [9]:
## insert yesterday reporting to bdd
# try:
#     data_job = data_up_pip(rapport) 
#     data_job.up_rapp()
#     data_job.clean_leave()
# except:
#     print("BDD job Failed -- Normal if you not have acces to main code")